In [1]:
import torch
print(torch.cuda.is_available())

True


In [2]:
import pandas as pd
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import DistilBertForMultipleChoice, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
import os

/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py:184: UserWarning: 
    Found GPU0 NVIDIA GeForce GTX 950M which is of cuda capability 5.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability supported by this library is 5.2.
    
  warnings.warn(


In [3]:
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [4]:
path = "../../data/processed"
dataset = load_dataset(path)
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'encoded_text'],
        num_rows: 86914
    })
    test: Dataset({
        features: ['text', 'encoded_text'],
        num_rows: 15338
    })
})

In [5]:
print(dataset["train"][0])

{'text': 'Malec skóroval padáčkem jako Poborský. Zase střílí góly až v play off V\xa0základní části hokejové extraligy nic, tam brněnský obránce Tomáš\xa0Malec\xa0góly nedává. Naposled se během dvaapadesátikolového maratonu trefil v\xa0sezoně 2013/14. V\xa0play off je to u něho o něčem jiném, tam se slovenský poctivec s\xa0číslem 71 prosazuje každoročně. Povedlo se mu to i letos. Gólem vteřinu před koncem prvního finále v\xa0Liberci poslal Kometu do vedení na 3:2, navíc to byl gól hodně kuriózní.', 'encoded_text': 13}


In [6]:
MODEL_NAME = "distilbert-base-uncased"

In [7]:
bitsandbytes_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enable 4-bit quantization
    bnb_4bit_quant_type="fp4",  # Use FP4 quantization
    bnb_4bit_compute_dtype=torch.float16,  # Set computation data type to float16 for reduced memory usage and potentially faster computation
    llm_int8_enable_fp32_cpu_offload=False,  # Disable CPU offload for INT8 operations
    llm_int8_has_fp16_weight=True  # Use FP16 weights for better fine-tuning stability
)

In [8]:
df = dataset['train'].to_pandas()
count_of_categories = df['encoded_text'].nunique()
print(count_of_categories)

24


In [9]:
model = DistilBertForMultipleChoice.from_pretrained(
    MODEL_NAME,
    quantization_config=bitsandbytes_config,
    attn_implementation="flash_attention_2",
    num_labels=count_of_categories
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


RuntimeError: CUDA error: no kernel image is available for execution on the device
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
